In [ ]:
from DeepFMKit.core import DeepFitFramework
from DeepFMKit.plotting import default_rc
from DeepFMKit.experiments import Experiment
import numpy as np

import matplotlib.pyplot as plt
plt.rcParams.update(default_rc)

In [ ]:
dff = DeepFitFramework(raw_file='../test/raw_data.txt', raw_labels=['raw1'])
dff.fit('raw1', n=20)

In [ ]:
axes = dff.plot()
plt.show()

In [ ]:
ax = dff.raws['raw1'].plot(xrange=[0,0.01])
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import constants as sc

from DeepFMKit import physics
from DeepFMKit.experiments import Experiment 

def create_phase_sweep_configs(params: dict) -> dict:
    """
    Takes a dictionary of trial parameters and returns the fully configured
    physics objects for the 2D phase sweep experiment.
    """
    # Get parameters for this specific trial
    m_target = params['m_target']
    opd = params['opd']
    psi = params['psi']
    ifo_phi = params['ifo_phi']

    # Create and configure the physics objects
    laser_config = physics.LaserConfig()
    main_ifo_config = physics.InterferometerConfig()

    laser_config.df = (m_target * sc.c) / (2 * np.pi * opd)
    laser_config.psi = psi
    main_ifo_config.phi = ifo_phi
    
    # Return the configured objects in a dictionary
    return {
        'laser_config': laser_config,
        'main_ifo_config': main_ifo_config
    }

# --- 1. Declaratively Define the Experiment ---
exp = Experiment(description="2D Sweep of Modulation and IFO Phase")

# Define the two axes to sweep
exp.add_axis('psi', np.linspace(-np.pi, np.pi, 25))
exp.add_axis('ifo_phi', np.linspace(-np.pi, np.pi, 25))

# Define fixed physical parameters
exp.set_static({'m_target': 6.0, 'opd': 0.2})

# Tell the experiment which function to use to create the physics configs
exp.set_config_factory(create_phase_sweep_configs)

# Define the analysis to run. All standard fit parameters will be extracted.
exp.add_analysis(
    name='nls_fit',
    fitter_method='nls',
    fitter_kwargs={'ndata': 15, 'parallel': False}
)

print("Experiment configured successfully.")

# --- 2. Run the Experiment ---
# The .run() method handles all the parallelization and result aggregation.
if __name__ == "__main__":
    results = exp.run()

In [ ]:
# The built-in .plot() method is perfect for this standard 2D analysis
ax = exp.plot(analysis_name='nls_fit', param_to_plot='phi', stat='mean')

# Customize the labels for publication-quality output
ax.set_xlabel(r'Input Modulation Phase, $\psi_{\rm in}$ (rad)')
ax.set_ylabel(r'Input IFO Phase, $\phi_{\rm in}$ (rad)')

# Access the colorbar axis to set its label
cbar_ax = ax.figure.get_axes()[1]
cbar_ax.set_ylabel(r'Fitted IFO Phase, $\hat{\phi}$ (rad)', fontsize=14)

plt.show()